Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (18724, 28, 28), (18724,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (18724, 784), (18724, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### logistic regression with l2 loss function

regularizing with beta = **0.01**


In [5]:
# multinomial logistic regression 
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])

        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels) )

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 46.821888
Training accuracy: 11.7%
Validation accuracy: 14.8%
Loss at step 100: 12.265929
Training accuracy: 74.8%
Validation accuracy: 72.5%
Loss at step 200: 4.623464
Training accuracy: 79.8%
Validation accuracy: 76.0%
Loss at step 300: 2.015288
Training accuracy: 82.6%
Validation accuracy: 78.8%
Loss at step 400: 1.130751
Training accuracy: 84.0%
Validation accuracy: 80.7%
Loss at step 500: 0.827437
Training accuracy: 84.5%
Validation accuracy: 81.5%
Loss at step 600: 0.722029
Training accuracy: 84.6%
Validation accuracy: 81.7%
Loss at step 700: 0.685019
Training accuracy: 84.8%
Validation accuracy: 81.8%
Loss at step 800: 0.671914
Training accuracy: 84.8%
Validation accuracy: 81.9%
Test accuracy: 88.9%


### neural network with l2 loss function

In [7]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.648849
Minibatch accuracy: 10.2%
Validation accuracy: 14.4%
Minibatch loss at step 500: 0.928264
Minibatch accuracy: 78.9%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 0.941039
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 0.899968
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 0.781843
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.741092
Minibatch accuracy: 82.8%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.795314
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Test accuracy: 88.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### log reg with l2 loss and small training data

In [22]:
# multinomial logistic regression 
train_subset = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [23]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])

        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels) )

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 49.642525
Training accuracy: 6.0%
Validation accuracy: 8.6%
Loss at step 100: 10.987138
Training accuracy: 100.0%
Validation accuracy: 52.1%
Loss at step 200: 4.085679
Training accuracy: 100.0%
Validation accuracy: 58.4%
Loss at step 300: 1.562234
Training accuracy: 100.0%
Validation accuracy: 64.1%
Loss at step 400: 0.640979
Training accuracy: 100.0%
Validation accuracy: 68.5%
Loss at step 500: 0.304736
Training accuracy: 100.0%
Validation accuracy: 70.7%
Loss at step 600: 0.181831
Training accuracy: 100.0%
Validation accuracy: 71.8%
Loss at step 700: 0.136748
Training accuracy: 100.0%
Validation accuracy: 72.1%
Loss at step 800: 0.120091
Training accuracy: 100.0%
Validation accuracy: 72.3%
Test accuracy: 80.3%


Training accuracy goes to 100% due to the small sample size, however, the model is not as good the validation samples.

### neural network wiht l2 loss and small training data

In [35]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [36]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.222691
Minibatch accuracy: 8.6%
Validation accuracy: 12.0%
Minibatch loss at step 500: 0.937366
Minibatch accuracy: 79.7%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 0.940764
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.899765
Minibatch accuracy: 79.7%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.781850
Minibatch accuracy: 80.5%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.741092
Minibatch accuracy: 82.8%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.795301
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Test accuracy: 88.4%


In [37]:
num_steps = 3001

train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.562416
Minibatch accuracy: 14.1%
Validation accuracy: 12.8%
Minibatch loss at step 500: 0.488236
Minibatch accuracy: 97.7%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 0.281167
Minibatch accuracy: 99.2%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 0.256229
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 0.251649
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2500: 0.250091
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.252528
Minibatch accuracy: 100.0%
Validation accuracy: 77.4%
Test accuracy: 84.9%


We see overfitting occuring on neural network model when the training set is 3 times smaller.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [38]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [39]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 449.394684
Minibatch accuracy: 10.2%
Validation accuracy: 32.0%
Minibatch loss at step 500: 31.107643
Minibatch accuracy: 71.1%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 18.377457
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 9.970980
Minibatch accuracy: 71.9%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 3.690306
Minibatch accuracy: 75.8%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 6.619049
Minibatch accuracy: 68.8%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 7.998204
Minibatch accuracy: 78.1%
Validation accuracy: 79.3%
Test accuracy: 87.1%


In [40]:
num_steps = 3001

train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 488.167145
Minibatch accuracy: 13.3%
Validation accuracy: 44.1%
Minibatch loss at step 500: 4.104881
Minibatch accuracy: 98.4%
Validation accuracy: 77.1%
Minibatch loss at step 1000: 1.809182
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 1.129296
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 0.694688
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.933634
Minibatch accuracy: 99.2%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 0.346199
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Test accuracy: 84.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### trying smaller beta and bigger num steps

In [43]:
batch_size = 128
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [44]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 554.648987
Minibatch accuracy: 9.4%
Validation accuracy: 31.3%
Minibatch loss at step 500: 16.525526
Minibatch accuracy: 70.3%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 20.638678
Minibatch accuracy: 72.7%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 12.109427
Minibatch accuracy: 75.8%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 6.230000
Minibatch accuracy: 65.6%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 7.725407
Minibatch accuracy: 71.1%
Validation accuracy: 78.4%
Minibatch loss at step 3000: 5.457012
Minibatch accuracy: 75.8%
Validation accuracy: 79.2%
Minibatch loss at step 3500: 4.158463
Minibatch accuracy: 75.8%
Validation accuracy: 79.7%
Minibatch loss at step 4000: 6.545417
Minibatch accuracy: 76.6%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 4.633209
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 5000: 2.509747
Minibatch accuracy: 82.0%
Validation acc

### trying keep prob of 0.1

In [48]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.1}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1055.875244
Minibatch accuracy: 11.7%
Validation accuracy: 32.1%
Minibatch loss at step 500: 219590.375000
Minibatch accuracy: 44.5%
Validation accuracy: 71.2%
Minibatch loss at step 1000: 21063032.000000
Minibatch accuracy: 46.9%
Validation accuracy: 70.8%
Minibatch loss at step 1500: 417204384.000000
Minibatch accuracy: 39.8%
Validation accuracy: 67.3%
Minibatch loss at step 2000: 11588214784.000000
Minibatch accuracy: 35.2%
Validation accuracy: 66.1%
Minibatch loss at step 2500: 123983814656.000000
Minibatch accuracy: 40.6%
Validation accuracy: 63.4%
Minibatch loss at step 3000: 5050337853440.000000
Minibatch accuracy: 43.8%
Validation accuracy: 69.4%
Test accuracy: 75.9%


### trying with learning rate and multiple keep probs

In [55]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [56]:
num_steps = 3001

for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    print ">>with keep prob of " + str(kp)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp}
            _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

>>with keep prob of0.5
Initialized
Minibatch loss at step 0: 476.243591
Minibatch accuracy: 10.9%
Validation accuracy: 34.6%
Minibatch loss at step 500: 29.974281
Minibatch accuracy: 72.7%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 18.089090
Minibatch accuracy: 74.2%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 12.756054
Minibatch accuracy: 71.1%
Validation accuracy: 78.1%
Minibatch loss at step 2000: 9.580293
Minibatch accuracy: 66.4%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 6.495479
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 5.315317
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Test accuracy: 87.0%
>>with keep prob of0.6
Initialized
Minibatch loss at step 0: 433.451294
Minibatch accuracy: 14.1%
Validation accuracy: 33.5%
Minibatch loss at step 500: 33.311592
Minibatch accuracy: 71.1%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 14.231369
Minibatch accuracy: 79.7%
Validation accuracy: 79.0

### trying different learning rates

In [66]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)

    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu, weights) + biases) 

    test_relu = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights) + biases)

In [68]:
for lr in np.arange(0.0001, 0.001, 0.0001).tolist():

    print '>>trying lr of ' + str(lr)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, learnr : lr}
            _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

trying lr of 0.0001
Initialized
Minibatch loss at step 0: 332.018372
Minibatch accuracy: 10.9%
Validation accuracy: 33.7%
Minibatch loss at step 500: 11.340741
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 12.299367
Minibatch accuracy: 78.9%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 5.031932
Minibatch accuracy: 79.7%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 2.718194
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 2500: 6.110866
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 4.337133
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Test accuracy: 88.4%
trying lr of 0.0002
Initialized
Minibatch loss at step 0: 304.924866
Minibatch accuracy: 13.3%
Validation accuracy: 31.3%
Minibatch loss at step 500: 10.659449
Minibatch accuracy: 85.9%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 9.710072
Minibatch accuracy: 76.6%
Validation accuracy: 79.0%
Miniba

best learning rate is around *0.0003* and *0.0004*

### trying 2 layer neural network

In [100]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable( tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable( tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu( tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(  tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu( tf.matmul( test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)


In [101]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0, learnr : 0.001}
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4536.678223
Minibatch accuracy: 11.7%
Validation accuracy: 13.0%
Minibatch loss at step 500: 394.217407
Minibatch accuracy: 73.4%
Validation accuracy: 76.0%
Minibatch loss at step 1000: 340.395996
Minibatch accuracy: 78.1%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 320.985291
Minibatch accuracy: 75.8%
Validation accuracy: 78.2%
Minibatch loss at step 2000: 161.737305
Minibatch accuracy: 77.3%
Validation accuracy: 78.8%
Minibatch loss at step 2500: 161.635468
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 171.968903
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Test accuracy: 87.4%


In [103]:
batch_size = 128
beta = 0.001

hidden_nodes1 = 1024
hidden_nodes2 = 512

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # new hidden layer 1
    
    hidden_weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes1]) )
    hidden_biases = tf.Variable( tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu( tf.matmul( tf_train_dataset, hidden_weights) + hidden_biases)
    
    # add dropout on hidden layer
    keep_prob = tf.placeholder("float")
    hidden_layer_drop = tf.nn.dropout(hidden_layer, keep_prob)
    
    # new hidden layer 2
    hidden_weights2 = tf.Variable( tf.truncated_normal([hidden_nodes1, hidden_nodes2]) )
    hidden_biases2 = tf.Variable( tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu( tf.matmul( hidden_layer_drop, hidden_weights2) + hidden_biases2)
    
    # add dropout on hidden layer
    hidden_layer_drop2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    # Variables.
    weights = tf.Variable( tf.truncated_normal([hidden_nodes2, num_labels])) 
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer_drop2, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learnr = tf.placeholder("float")
    learning_rate = tf.train.exponential_decay(learnr, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step= global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)    
    
    valid_relu1 = tf.nn.relu(  tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)    
    valid_relu2 = tf.nn.relu(  tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)    
    
    valid_prediction = tf.nn.softmax( tf.matmul(valid_relu2, weights) + biases) 
    
    test_relu1 = tf.nn.relu( tf.matmul( tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu( tf.matmul( test_relu1, hidden_weights2) + hidden_biases2)   
    
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)

In [ ]:
num_steps = 3001
scores = {}
for kp in [0.5, 0.6, 0.7, 0.9, 0.9, 1.0]:
    for lr in np.arange(0.0001, 0.001, 0.0001).tolist():
        print ">> with keep prob of " + str(kp)
        print ">> with lr  " + str(lr)
        with tf.Session(graph=graph) as session:
          tf.initialize_all_variables().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : kp, learnr : lr}
            _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict )
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
          acc = accuracy(test_prediction.eval(), test_labels)
          scores[(kp, lr)] = acc
          print("Test accuracy: %.1f%%" % acc)
print scores

>> with keep prob of 0.5
>> with lr  0.0001
Initialized
Minibatch loss at step 0: 10156.047852
Minibatch accuracy: 7.0%
Validation accuracy: 12.2%
Minibatch loss at step 500: 3440.179932
Minibatch accuracy: 45.3%
Validation accuracy: 71.9%
Minibatch loss at step 1000: 2197.126953
Minibatch accuracy: 51.6%
Validation accuracy: 76.2%
Minibatch loss at step 1500: 1542.451172
Minibatch accuracy: 60.2%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 1624.985962
Minibatch accuracy: 58.6%
Validation accuracy: 78.6%
Minibatch loss at step 2500: 1524.648560
Minibatch accuracy: 57.0%
Validation accuracy: 79.1%
Minibatch loss at step 3000: 1360.887207
Minibatch accuracy: 62.5%
Validation accuracy: 79.6%
Test accuracy: 87.2%
>> with keep prob of 0.5
>> with lr  0.0002
Initialized
Minibatch loss at step 0: 11233.877930
Minibatch accuracy: 4.7%
Validation accuracy: 8.0%
Minibatch loss at step 500: 2444.718994
Minibatch accuracy: 52.3%
Validation accuracy: 74.6%
Minibatch loss at step 1000: 1